In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
df = pd.read_csv('diabetes.csv')

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()["Outcome"] #btaye ga outcome kis column pr zada depend kr rha hai

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
x = df.iloc[:,:-1]
y = df["Outcome"]

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [9]:
df.shape

(768, 9)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size= 0.2,random_state = 42)

In [60]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [14]:
model= Sequential()
model.add(Dense(32,activation = 'relu',input_dim = 8))
model.add(Dense(1,activation ='sigmoid'))

C:\Program Files\Python310\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.compile(optimizer = 'Adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [23]:
model.fit(x_train,y_train,batch_size = 32 , epochs = 100,validation_data = (x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7883 - loss: 0.4371 - val_accuracy: 0.7792 - val_loss: 0.5057
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7850 - loss: 0.4353 - val_accuracy: 0.7662 - val_loss: 0.5076
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7866 - loss: 0.4336 - val_accuracy: 0.7727 - val_loss: 0.5089
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7866 - loss: 0.4330 - val_accuracy: 0.7727 - val_loss: 0.5091
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7883 - loss: 0.4316 - val_accuracy: 0.7792 - val_loss: 0.5094
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7932 - loss: 0.4309 - val_accuracy: 0.7727 - val_loss: 0.5106
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7932 - loss: 0.4301 - val_accuracy: 0.7727 - val_loss: 0.5103
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7948 - loss: 0.4293 - val_accuracy: 0.7

In [3]:
import keras_tuner as kt

# find out best optimizer

#### aik function bnana hai build model k naam se jis mein hp (hyper parameter ko pass krein gy ) ye aur is function mein model scratch se build aur compile ho ga.


In [38]:
def build_model(hp):
    model= Sequential()
    model.add(Dense(32,activation = 'relu',input_dim = 8))
    model.add(Dense(1,activation ='sigmoid'))
    optimizer = hp.Choice('optimizer' , values = ['sgd','adam','adadelta','rmsprop']) # ab 4 models bnein gy hrr aik optimizer k sath 1 model ho ga
    model.compile(optimizer = optimizer , loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model # jo model build ho ga wo return ho ga

#### Now create a tuner object that takes 3 parameters: 
#### 1- function name -- jo bnaya ho ga
#### 2- objective -- kis cheez ko brhana ya ghtana hai ye khud identify kre ga k kmm krna hai ya zada
#### 3- max_trials -- kitni baar check krna hai

In [39]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5)

Reloading Tuner from .\untitled_project\tuner0.json


In [46]:
tuner.search(x_train,y_train,epochs = 5,validation_data = (x_test,y_test))  # ye hrr otptimizer ka model build kre ga aur accuracy find kre ga
tuner.results_summary()  # training ki summary show krwaye ga

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
optimizer: adam
Score: 0.8246753215789795

Trial 0 summary
Hyperparameters:
optimizer: sgd
Score: 0.7272727489471436

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7077922224998474

Trial 3 summary
Hyperparameters:
optimizer: adadelta
Score: 0.38311687111854553


In [43]:
tuner.get_best_hyperparameters()[0].values # wo hyperparameter de ga jis prr best results aa rhy hain

{'optimizer': 'adam'}

In [45]:
tuner.results_summary()  

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
optimizer: adam
Score: 0.8246753215789795

Trial 0 summary
Hyperparameters:
optimizer: sgd
Score: 0.7272727489471436

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7077922224998474

Trial 3 summary
Hyperparameters:
optimizer: adadelta
Score: 0.38311687111854553


In [48]:
model = tuner.get_best_models(num_models=1)[0] # ye wo model nikale ga jo best perform kr rha hao ga

In [49]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.fit(x_train,y_train,batch_size = 32,epochs = 100 , initial_epoch = 6,validation_data = (x_test,y_test)) # ye model 5 epochs tk phle train ho chuka hai kyu k is ko Random Search se extract kya hai is lye ye 6 epochs se start ho ga

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7476 - loss: 0.5348 - val_accuracy: 0.8117 - val_loss: 0.5298
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7622 - loss: 0.5109 - val_accuracy: 0.7857 - val_loss: 0.5153
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7606 - loss: 0.4953 - val_accuracy: 0.7857 - val_loss: 0.5065
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7638 - loss: 0.4835 - val_accuracy: 0.7857 - val_loss: 0.4987
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7687 - loss: 0.4750 - val_accuracy: 0.7857 - val_loss: 0.4955
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7720 - loss: 0.4684 - val_accuracy: 0.7792 - val_loss: 0.4958
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7671 - loss: 0.4636 - val_accuracy: 0.7857 - val_loss: 0.4941
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7704 - loss: 0.4592 - val_accuracy: 0.78

# Find number of nerons in particular layer

In [18]:
def build_model(hp):
    model= Sequential()
    units = hp.Int('units',min_value = 8,max_value = 128,step = 8) # starting pont = 8 , ending = 128 , step size = 8
    # is ne alg alg models build kren hain phle mein 8 nodes hon gy doosre mein 16 hon gy .. aaese krty krty last wale mein 128 hon gy
    model.add(Dense(units = units ,activation = 'relu',input_dim = 8))
    model.add(Dense(1,activation ='sigmoid'))
    model.compile(optimizer = 'adam' , loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model # jo model build ho ga wo return ho ga

In [21]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5,directory = 'mydir',project_name = 'project1') # is ko run krne se aik json file bne gy .. jis ko hmm apni directory aur project ka name k sath store krewa skty hain

C:\Program Files\Python310\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
tuner.search(x_train,y_train,epochs = 5,validation_data = (x_test,y_test))  # ye hrr otptimizer ka model build kre ga aur accuracy find kre ga
tuner.results_summary()  

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 21s
Results summary
Results in mydir\project1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 120
Score: 0.7857142686843872

Trial 1 summary
Hyperparameters:
units: 104
Score: 0.7727272510528564

Trial 2 summary
Hyperparameters:
units: 112
Score: 0.7207792401313782

Trial 4 summary
Hyperparameters:
units: 40
Score: 0.7142857313156128

Trial 0 summary
Hyperparameters:
units: 32
Score: 0.6428571343421936


In [23]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [24]:
model = tuner.get_best_models(num_models=1)[0] 
model.summary()

C:\Program Files\Python310\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 120)                 │           1,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             121 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,201 (4.69 KB)

 Trainable params: 1,201 (4.69 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(x_train,y_train,batch_size = 32,epochs = 100 , initial_epoch = 6,validation_data = (x_test,y_test)) # ye model 5 epochs tk phle train ho chuka hai kyu k is ko Random Search se extract kya hai is lye ye 6 epochs se start ho ga

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7671 - loss: 0.4774 - val_accuracy: 0.7662 - val_loss: 0.5042
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7769 - loss: 0.4626 - val_accuracy: 0.7662 - val_loss: 0.4991
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7785 - loss: 0.4554 - val_accuracy: 0.7792 - val_loss: 0.4983
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7850 - loss: 0.4488 - val_accuracy: 0.7662 - val_loss: 0.5034
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7850 - loss: 0.4462 - val_accuracy: 0.7532 - val_loss: 0.5048
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7932 - loss: 0.4414 - val_accuracy: 0.7597 - val_loss: 0.5071
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7915 - loss: 0.4386 - val_accuracy: 0.7532 - val_loss: 0.5107
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7899 - loss: 0.4373 - val_accurac

# Select Number of layers

In [27]:
def build_model(hp):
    model= Sequential()
    model.add(Dense(72 ,activation = 'relu',input_dim = 8)) # first hidden layer
    for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):
        model.add(Dense(72,activation = 'relu'))
    # phli bar 1 layer bne gy 
    # doosri baar 2 layers bnein gy .. aese krt krty 10 layers tk ka model bne ga
    model.add(Dense(1,activation ='sigmoid')) # output layer
    model.compile(optimizer = 'adam' , loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model # jo model build ho ga wo return ho ga

In [28]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5,directory = 'mydir',project_name = 'project2') # is ko run krne se aik json file bne gy .. jis ko hmm apni directory aur project ka name k sath store krewa skty hain

In [29]:
tuner.search(x_train,y_train,epochs = 5,validation_data = (x_test,y_test))  # ye hrr otptimizer ka model build kre ga aur accuracy find kre ga
tuner.results_summary()  

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 27s
Results summary
Results in mydir\project2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 3
Score: 0.7857142686843872

Trial 4 summary
Hyperparameters:
num_layers: 5
Score: 0.7792207598686218

Trial 1 summary
Hyperparameters:
num_layers: 10
Score: 0.7727272510528564

Trial 2 summary
Hyperparameters:
num_layers: 2
Score: 0.7597402334213257

Trial 3 summary
Hyperparameters:
num_layers: 9
Score: 0.7467532753944397


In [30]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [31]:
model = tuner.get_best_models(num_models=1)[0] 
model.summary()

C:\Program Files\Python310\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 72)                  │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              73 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,489 (64.41 KB)

 Trainable params: 16,489 (64.41 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model.fit(x_train,y_train,batch_size = 32,epochs = 100 , initial_epoch = 6,validation_data = (x_test,y_test)) 

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7736 - loss: 0.4724 - val_accuracy: 0.7792 - val_loss: 0.4906
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7850 - loss: 0.4474 - val_accuracy: 0.7597 - val_loss: 0.5033
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7997 - loss: 0.4278 - val_accuracy: 0.7597 - val_loss: 0.5191
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7980 - loss: 0.4219 - val_accuracy: 0.7597 - val_loss: 0.5135
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8029 - loss: 0.4120 - val_accuracy: 0.7403 - val_loss: 0.5281
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8127 - loss: 0.3973 - val_accuracy: 0.7468 - val_loss: 0.5298
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8192 - loss: 0.3821 - val_accuracy: 0.7532 - val_loss: 0.5280
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8306 - loss: 0.3816 - val_accurac

# All in one code

In [54]:
def build_model(hp):
    model = Sequential()
    counter = 0 # to track layer number 

    for i in range (hp.Int('layer_num',min_value = 1,max_value = 10)):

        if counter == 0: # for first hidden layer
            model.add(Dense(units = hp.Int('units' + str(i),min_value = 8,max_value = 128,step=8), 
                            activation = hp.Choice('Activation' + str(i) , values = ['relu','tanh','sigmoid']),
                            input_dim = 8))
        else:
             model.add(Dense(units = hp.Int('units' + str(i),min_value = 8,max_value = 128,step=8), 
                            activation = hp.Choice('Activation' + str(i) , values = ['relu','tanh','sigmoid'])
                            ))
        counter +=1
    model.add(Dense(1,activation = 'sigmoid')) # output LAyer
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop']), loss = 'binary_crossentropy' ,metrics = ['accuracy'])
    return model
                        

In [55]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5,directory = 'mydir',project_name = 'project3') # is ko run krne se aik json file bne gy .. jis ko hmm apni directory aur project ka name k sath store krewa skty hain

Reloading Tuner from mydir\project3\tuner0.json


In [56]:
tuner.search(x_train,y_train,epochs = 5,validation_data = (x_test,y_test))  # ye hrr otptimizer ka model build kre ga aur accuracy find kre ga
tuner.results_summary()  

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6558441519737244

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 06m 40s
Results summary
Results in mydir\project3
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
layer_num: 4
units0: 32
Activation0: relu
optimizer: adam
units1: 96
Activation1: tanh
units2: 40
Activation2: relu
units3: 8
Activation3: tanh
units4: 128
Activation4: sigmoid
Score: 0.7662337422370911

Trial 4 summary
Hyperparameters:
layer_num: 1
units0: 16
Activation0: tanh
optimizer: rmsprop
units1: 88
Activation1: relu
units2: 72
Activation2: relu
units3: 16
Activation3: tanh
units4: 24
Activation4: sigmoid
units5: 48
Activation5: relu
units6: 32
Activation6: sigmoid
Score: 0.6558441519737244

Trial 3 summary
Hyperparameters:
layer_num: 7
units0: 88
Activation0: tanh
optimizer: sgd
units1: 112
Activation1: sigmoid
units2: 80
Activation2: relu
units3: 120
Activation3: tanh
units4: 80
Activation4: 

In [57]:
tuner.get_best_hyperparameters()[0].values

{'layer_num': 4,
 'units0': 32,
 'Activation0': 'relu',
 'optimizer': 'adam',
 'units1': 96,
 'Activation1': 'tanh',
 'units2': 40,
 'Activation2': 'relu',
 'units3': 8,
 'Activation3': 'tanh',
 'units4': 128,
 'Activation4': 'sigmoid'}

In [58]:
model = tuner.get_best_models(num_models=1)[0] 
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 96)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 40)                  │           3,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │             328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,673 (29.97 KB)

 Trainable params: 7,673 (29.97 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model.fit(x_train,y_train,batch_size = 32,epochs = 200 , initial_epoch = 6,validation_data = (x_test,y_test)) 

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7720 - loss: 0.4626 - val_accuracy: 0.7468 - val_loss: 0.5033
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7785 - loss: 0.4512 - val_accuracy: 0.7468 - val_loss: 0.4999
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7769 - loss: 0.4413 - val_accuracy: 0.7857 - val_loss: 0.4907
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7818 - loss: 0.4348 - val_accuracy: 0.7792 - val_loss: 0.4830
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7980 - loss: 0.4320 - val_accuracy: 0.7727 - val_loss: 0.4922
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7948 - loss: 0.4281 - val_accuracy: 0.7597 - val_loss: 0.4995
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7997 - loss: 0.4199 - val_accuracy: 0.7532 - val_loss: 0.5081
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7997 - loss: 0.4158 - val_accurac

# We can use drop out in it also 

In [63]:
def build_model(hp):
    model = Sequential()
    counter = 0 # to track layer number 

    for i in range (hp.Int('layer_num',min_value = 1,max_value = 10)):

        if counter == 0: # for first hidden layer
            model.add(Dense(units = hp.Int('units' + str(i),min_value = 8,max_value = 128,step=8), 
                            activation = hp.Choice('Activation' + str(i) , values = ['relu','tanh','sigmoid']),
                            input_dim = 8))
            model.add(Dropout(hp.Choice('dropout'+str(i) ,values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
             model.add(Dense(units = hp.Int('units' + str(i),min_value = 8,max_value = 128,step=8), 
                            activation = hp.Choice('Activation' + str(i) , values = ['relu','tanh','sigmoid'])
                            ))
             model.add(Dropout(hp.Choice('dropout'+str(i) ,values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter +=1
    model.add(Dense(1,activation = 'sigmoid')) # output LAyer
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop']), loss = 'binary_crossentropy' ,metrics = ['accuracy'])
    return model

In [64]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5,directory = 'mydir',project_name = 'project3') # is ko run krne se aik json file bne gy .. jis ko hmm apni directory aur project ka name k sath store krewa skty hain

Reloading Tuner from mydir\project3\tuner0.json


In [65]:
tuner.search(x_train,y_train,epochs = 5,validation_data = (x_test,y_test))  # ye hrr otptimizer ka model build kre ga aur accuracy find kre ga
tuner.results_summary()  

Results summary
Results in mydir\project3
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
layer_num: 4
units0: 32
Activation0: relu
optimizer: adam
units1: 96
Activation1: tanh
units2: 40
Activation2: relu
units3: 8
Activation3: tanh
units4: 128
Activation4: sigmoid
Score: 0.7662337422370911

Trial 4 summary
Hyperparameters:
layer_num: 1
units0: 16
Activation0: tanh
optimizer: rmsprop
units1: 88
Activation1: relu
units2: 72
Activation2: relu
units3: 16
Activation3: tanh
units4: 24
Activation4: sigmoid
units5: 48
Activation5: relu
units6: 32
Activation6: sigmoid
Score: 0.6558441519737244

Trial 3 summary
Hyperparameters:
layer_num: 7
units0: 88
Activation0: tanh
optimizer: sgd
units1: 112
Activation1: sigmoid
units2: 80
Activation2: relu
units3: 120
Activation3: tanh
units4: 80
Activation4: relu
units5: 8
Activation5: relu
units6: 8
Activation6: relu
Score: 0.649350643157959

Trial 1 summary
Hyperparameters:
layer_num: 5
units0: 2

In [66]:
tuner.get_best_hyperparameters()[0].values

{'layer_num': 4,
 'units0': 32,
 'Activation0': 'relu',
 'optimizer': 'adam',
 'units1': 96,
 'Activation1': 'tanh',
 'units2': 40,
 'Activation2': 'relu',
 'units3': 8,
 'Activation3': 'tanh',
 'units4': 128,
 'Activation4': 'sigmoid'}

In [67]:
model = tuner.get_best_models(num_models=1)[0] 
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 96)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 96)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 40)                  │           3,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 40)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │             328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,673 (29.97 KB)

 Trainable params: 7,673 (29.97 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.fit(x_train,y_train,batch_size = 32,epochs = 200 , initial_epoch = 6,validation_data = (x_test,y_test)) 

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7590 - loss: 0.4849 - val_accuracy: 0.7532 - val_loss: 0.5057
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7752 - loss: 0.4774 - val_accuracy: 0.7403 - val_loss: 0.5074
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7622 - loss: 0.4655 - val_accuracy: 0.7597 - val_loss: 0.4969
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7785 - loss: 0.4561 - val_accuracy: 0.7662 - val_loss: 0.4979
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7687 - loss: 0.4565 - val_accuracy: 0.7662 - val_loss: 0.4995
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7752 - loss: 0.4587 - val_accuracy: 0.7468 - val_loss: 0.4999
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7850 - loss: 0.4424 - val_accuracy: 0.7468 - val_loss: 0.5054
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7834 - loss: 0.4458 - val_accurac